In [1]:
import numpy as np
import pandas as pd
import nltk 
import seaborn as sns 
import matplotlib.pyplot as plt 
from transformers import BertModel
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [4]:
import pickle
pickle_path = r'C:\Users\Sahan\Documents\Phase_4_Project\Embeddings\bert_embeddings.pkl'

# Load the Pickle file
with open(pickle_path, 'rb') as file:
    embeddings = pickle.load(file)

# If the embeddings are a dictionary or list, convert to a DataFrame for easier manipulation
df_embeddings = pd.DataFrame(embeddings)

# Check the first few rows of the dataframe
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.231322,0.306714,0.367601,0.030625,0.664075,-0.224980,0.048418,0.221035,-0.200895,-0.229604,...,0.036324,-0.092945,-0.047764,-0.026186,0.295380,0.003586,-0.450574,-0.126786,-0.315632,-0.051949
1,0.376771,0.328142,0.416568,0.115086,0.872759,-0.204387,0.003964,0.463500,-0.272053,-0.102894,...,-0.165577,-0.019288,-0.017614,-0.109886,0.150803,-0.079793,-0.448492,-0.099017,-0.352305,-0.034338
2,-0.133441,0.252630,0.364010,0.206061,0.828878,-0.320091,0.153737,0.317311,-0.282796,-0.409026,...,0.079692,-0.310298,0.163278,0.060609,0.273035,-0.018295,-0.571709,-0.089511,-0.180690,-0.086050
3,-0.243556,0.250552,0.373141,0.110616,0.398387,-0.238613,0.086858,0.194104,-0.101586,-0.266791,...,-0.011533,-0.335915,-0.004612,0.114166,0.311642,-0.102268,-0.621948,-0.055168,-0.310393,-0.145939
4,-0.192203,0.213387,0.313394,0.202181,0.683933,-0.242526,0.107760,0.289659,-0.147931,-0.313180,...,0.021700,-0.167761,0.225822,0.114646,0.305736,0.050840,-0.637124,-0.121972,-0.202731,-0.014809


In [5]:
og_df = pd.read_csv(r'C:\Users\Sahan\Documents\Phase_4_Project\CSV\US_Hotel_DB_2.csv')
og_df

,HotelName,HotelRating,Address,City,State,Facilities
0,Sunbelt Lodge Motel,2,1903 Veterans Memorial Drive AbbevilleLouisian...,Abbeville,Louisiana,shared loungetv area meetingbanquet facilities...
1,Budget Inn & Suites,2,2115 Charity Street AbbevilleLouisiana 70510,Abbeville,Louisiana,24hour front desk nonsmoking rooms laundry fre...
2,Best Western Abbeville Inn & Suites,2,3515 Veterans Memorial Dr Abbeville AbbevilleL...,Abbeville,Louisiana,wheelchair accessible may have limitations ven...
3,Motel 6 Abbeville LA,2,3523 Veterans Memorial Drive AbbevilleLouisian...,Abbeville,Louisiana,wheelchair accessible may have limitations whe...
4,Days Inn by Wyndham Abbeville,2,730 Veterans Memorial Dr AbbevilleLouisiana 70...,Abbeville,Louisiana,express checkout multilingual staff 24hour fro...
...,...,...,...,...,...,...
137943,Country Inn & Suites by Radisson Zion IL,2,1100 33Rd Street Zion 60099 ZionIllinois 60099,Zion,Illinois,dry cleaninglaundry service wheelchair accessi...
137944,The Inn at Market Square,3,2723 Sheridan Road ZionIllinois 60099,Zion,Illinois,gift shops or newsstand shopping on site propo...
137945,Best Western Plus Crossroads Inn & Suites,2,135 Wood Ridge Ter Zion Crossroads Zion Crossr...,Zion Crossroads,Virginia,daily safedeposit box at front desk smokefree ...
137946,Zion Ponderosa Ranch Resort,2,Twin Knolls Road Zion National Park Orderville...,Zion National Park,Utah,mountain biking on site atmbanking terrace zip...


In [6]:
og_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137948 entries, 0 to 137947
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   HotelName    137948 non-null  object
 1   HotelRating  137948 non-null  int64 
 2   Address      137948 non-null  object
 3   City         137948 non-null  object
 4   State        137948 non-null  object
 5   Facilities   137948 non-null  object
dtypes: int64(1), object(5)
memory usage: 6.3+ MB


In [7]:
# Strip extra spaces, convert to lowercase, and remove any special characters
og_df['HotelName_cleaned'] = og_df['HotelName'].str.strip().str.lower().str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

# Check the cleaned names
print(og_df[['HotelName', 'HotelName_cleaned']].head())


                             HotelName                   HotelName_cleaned
0                  Sunbelt Lodge Motel                 sunbelt lodge motel
1                  Budget Inn & Suites                  budget inn  suites
2  Best Western Abbeville Inn & Suites  best western abbeville inn  suites
3                 Motel 6 Abbeville LA                motel 6 abbeville la
4        Days Inn by Wyndham Abbeville       days inn by wyndham abbeville


In [8]:
import pandas as pd
import pickle


# Add HotelID to original DataFrame (use index as unique identifier)
og_df['HotelID'] = og_df.index

# Check the new df structure with HotelID
print(og_df[['HotelName', 'HotelRating', 'HotelID']].head())


                             HotelName  HotelRating  HotelID
0                  Sunbelt Lodge Motel            2        0
1                  Budget Inn & Suites            2        1
2  Best Western Abbeville Inn & Suites            2        2
3                 Motel 6 Abbeville LA            2        3
4        Days Inn by Wyndham Abbeville            2        4


In [9]:
# Load the embeddings from the pickle file
with open(r'C:\Users\Sahan\Documents\Phase_4_Project\Embeddings\bert_embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)

# Convert the embeddings into a DataFrame (assuming it's a list of 768-dimensional vectors)
embeddings_df = pd.DataFrame(embeddings)

# Add HotelID to the embeddings DataFrame (same index as the original DataFrame)
embeddings_df['HotelID'] = embeddings_df.index

# Check the embeddings DataFrame with HotelID
print(embeddings_df.head())


          0         1         2         3         4         5         6  \
0  0.231322  0.306714  0.367601  0.030625  0.664075 -0.224980  0.048418   
1  0.376771  0.328142  0.416568  0.115086  0.872759 -0.204387  0.003964   
2 -0.133441  0.252630  0.364010  0.206061  0.828878 -0.320091  0.153737   
3 -0.243556  0.250552  0.373141  0.110616  0.398387 -0.238613  0.086858   
4 -0.192203  0.213387  0.313394  0.202181  0.683933 -0.242526  0.107760   

          7         8         9  ...       759       760       761       762  \
0  0.221035 -0.200895 -0.229604  ... -0.092945 -0.047764 -0.026186  0.295380   
1  0.463500 -0.272053 -0.102894  ... -0.019288 -0.017614 -0.109886  0.150803   
2  0.317311 -0.282796 -0.409026  ... -0.310298  0.163278  0.060609  0.273035   
3  0.194104 -0.101586 -0.266791  ... -0.335915 -0.004612  0.114166  0.311642   
4  0.289659 -0.147931 -0.313180  ... -0.167761  0.225822  0.114646  0.305736   

        763       764       765       766       767  HotelID  
0  0.

In [10]:
# Merge the original DataFrame with embeddings on 'HotelID'
combined_df = pd.merge(og_df, embeddings_df, on='HotelID', how='inner')

# Check the combined DataFrame
combined_df

,HotelName,HotelRating,Address,City,State,Facilities,HotelName_cleaned,HotelID,0,1,...,758,759,760,761,762,763,764,765,766,767
0,Sunbelt Lodge Motel,2,1903 Veterans Memorial Drive AbbevilleLouisian...,Abbeville,Louisiana,shared loungetv area meetingbanquet facilities...,sunbelt lodge motel,0,0.231322,0.306714,...,0.036324,-0.092945,-0.047764,-0.026186,0.295380,0.003586,-0.450574,-0.126786,-0.315632,-0.051949
1,Budget Inn & Suites,2,2115 Charity Street AbbevilleLouisiana 70510,Abbeville,Louisiana,24hour front desk nonsmoking rooms laundry fre...,budget inn suites,1,0.376771,0.328142,...,-0.165577,-0.019288,-0.017614,-0.109886,0.150803,-0.079793,-0.448492,-0.099017,-0.352305,-0.034338
2,Best Western Abbeville Inn & Suites,2,3515 Veterans Memorial Dr Abbeville AbbevilleL...,Abbeville,Louisiana,wheelchair accessible may have limitations ven...,best western abbeville inn suites,2,-0.133441,0.252630,...,0.079692,-0.310298,0.163278,0.060609,0.273035,-0.018295,-0.571709,-0.089511,-0.180690,-0.086050
3,Motel 6 Abbeville LA,2,3523 Veterans Memorial Drive AbbevilleLouisian...,Abbeville,Louisiana,wheelchair accessible may have limitations whe...,motel 6 abbeville la,3,-0.243556,0.250552,...,-0.011533,-0.335915,-0.004612,0.114166,0.311642,-0.102268,-0.621948,-0.055168,-0.310393,-0.145939
4,Days Inn by Wyndham Abbeville,2,730 Veterans Memorial Dr AbbevilleLouisiana 70...,Abbeville,Louisiana,express checkout multilingual staff 24hour fro...,days inn by wyndham abbeville,4,-0.192203,0.213387,...,0.021700,-0.167761,0.225822,0.114646,0.305736,0.050840,-0.637124,-0.121972,-0.202731,-0.014809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137943,Country Inn & Suites by Radisson Zion IL,2,1100 33Rd Street Zion 60099 ZionIllinois 60099,Zion,Illinois,dry cleaninglaundry service wheelchair accessi...,country inn suites by radisson zion il,137943,-0.059963,0.352325,...,0.000135,-0.275857,0.035998,0.203938,0.387492,-0.033887,-0.652356,-0.067724,-0.287671,-0.192961
137944,The Inn at Market Square,3,2723 Sheridan Road ZionIllinois 60099,Zion,Illinois,gift shops or newsstand shopping on site propo...,the inn at market square,137944,-0.051890,0.336202,...,-0.193763,-0.401766,0.045899,0.184396,0.260074,-0.143118,-0.642698,0.040746,-0.247167,-0.282319
137945,Best Western Plus Crossroads Inn & Suites,2,135 Wood Ridge Ter Zion Crossroads Zion Crossr...,Zion Crossroads,Virginia,daily safedeposit box at front desk smokefree ...,best western plus crossroads inn suites,137945,-0.067061,0.402882,...,0.085916,-0.290551,0.132355,0.116010,0.359201,-0.028135,-0.565618,-0.131658,-0.184061,-0.089842
137946,Zion Ponderosa Ranch Resort,2,Twin Knolls Road Zion National Park Orderville...,Zion National Park,Utah,mountain biking on site atmbanking terrace zip...,zion ponderosa ranch resort,137946,0.182542,0.274810,...,-0.059132,-0.052606,-0.199437,-0.065881,0.213055,-0.189100,-0.461603,-0.010514,-0.175378,-0.205878


In [11]:

# Check the column names of the combined DataFrame
print(combined_df.columns)


Index([        'HotelName',       'HotelRating',           'Address',
                    'City',             'State',        'Facilities',
       'HotelName_cleaned',           'HotelID',                   0,
                         1,
       ...
                       758,                 759,                 760,
                       761,                 762,                 763,
                       764,                 765,                 766,
                       767],
      dtype='object', length=776)


In [12]:
# Check the first few rows to see the embedding columns
combined_df.head()


,HotelName,HotelRating,Address,City,State,Facilities,HotelName_cleaned,HotelID,0,1,...,758,759,760,761,762,763,764,765,766,767
0,Sunbelt Lodge Motel,2,1903 Veterans Memorial Drive AbbevilleLouisian...,Abbeville,Louisiana,shared loungetv area meetingbanquet facilities...,sunbelt lodge motel,0,0.231322,0.306714,...,0.036324,-0.092945,-0.047764,-0.026186,0.295380,0.003586,-0.450574,-0.126786,-0.315632,-0.051949
1,Budget Inn & Suites,2,2115 Charity Street AbbevilleLouisiana 70510,Abbeville,Louisiana,24hour front desk nonsmoking rooms laundry fre...,budget inn suites,1,0.376771,0.328142,...,-0.165577,-0.019288,-0.017614,-0.109886,0.150803,-0.079793,-0.448492,-0.099017,-0.352305,-0.034338
2,Best Western Abbeville Inn & Suites,2,3515 Veterans Memorial Dr Abbeville AbbevilleL...,Abbeville,Louisiana,wheelchair accessible may have limitations ven...,best western abbeville inn suites,2,-0.133441,0.252630,...,0.079692,-0.310298,0.163278,0.060609,0.273035,-0.018295,-0.571709,-0.089511,-0.180690,-0.086050
3,Motel 6 Abbeville LA,2,3523 Veterans Memorial Drive AbbevilleLouisian...,Abbeville,Louisiana,wheelchair accessible may have limitations whe...,motel 6 abbeville la,3,-0.243556,0.250552,...,-0.011533,-0.335915,-0.004612,0.114166,0.311642,-0.102268,-0.621948,-0.055168,-0.310393,-0.145939
4,Days Inn by Wyndham Abbeville,2,730 Veterans Memorial Dr AbbevilleLouisiana 70...,Abbeville,Louisiana,express checkout multilingual staff 24hour fro...,days inn by wyndham abbeville,4,-0.192203,0.213387,...,0.021700,-0.167761,0.225822,0.114646,0.305736,0.050840,-0.637124,-0.121972,-0.202731,-0.014809


In [13]:
# Step 1: Extract the embedding columns (from 8 to 775, which are columns 0 to 767)
embedding_columns = combined_df.columns[8:].tolist()  # These are columns 0 to 767
X_embeddings = combined_df[embedding_columns].values  # Extract embedding vectors


In [14]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Step 2: Standardize the embedding vectors (important for PCA)
scaler = StandardScaler()
X_embeddings_scaled = scaler.fit_transform(X_embeddings)

# Step 3: Apply PCA (e.g., reduce to 50 dimensions)
pca = PCA(n_components=50)
X_embeddings_pca = pca.fit_transform(X_embeddings_scaled)

# Now X_embeddings_pca contains the reduced representation of the embeddings


In [15]:
# Step 1: Separate target variable (HotelRating) and features (PCA embeddings + other metadata)
y = combined_df['HotelRating']  # Target: HotelRating
X = X_embeddings_pca  # Features: PCA embeddings


In [16]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Checking the shape of the data after the split
print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")


Training data shape: (110358, 50), Test data shape: (27590, 50)


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Step 2: Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 3: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 4: Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 0.2546220175146448
R-squared: 0.8135210750191735


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [3, 5, 7, 10, None],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],   # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],     # Minimum samples required at each leaf node
    'max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider when splitting
}

# Initialize RandomForestRegressor model
rf_model = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Score: {-grid_search.best_score_}")
